# 13.2_Project_Tracking_Subclasses_in_a_Framework

In [ ]:
from abc import ABC, abstractmethod

def print_word(word, formatter):  
    if not isinstance(formatter, WordFormatter):
        raise TypeError('formatter must be a WordFormatter')
        
    formatter.headings(word)

class WordFormatter(ABC):
    @abstractmethod
    def headings(self, word):
        pass
    
class UpperWordFormatter(WordFormatter):
    def headings(self, word):
        print(f'{word.upper()}')
        
class HTMLWordFormatter(WordFormatter):
    def headings(self, word):
        print(f'<html>{word}</html>')
        
class CSVWordFormatter(WordFormatter):
    def headings(self, word):
        print(f',{word},')

In [16]:
formatter = UpperWordFormatter()

In [17]:
print_word('hi', formatter)

HI


In [18]:
formatter = HTMLWordFormatter()

In [19]:
print_word('hi', formatter)

<html>hi</html>


In [20]:
formatter = CSVWordFormatter()

In [21]:
print_word('hi', formatter)

,hi,


## what happens if have a lot of formatter, will become hard to manage

#### create a factory function but has many if elses

In [26]:
from abc import ABC, abstractmethod

def print_word(word, formatter):  
    if not isinstance(formatter, WordFormatter):
        raise TypeError('formatter must be a WordFormatter')
        
    formatter.headings(word)

class WordFormatter(ABC):
    @abstractmethod
    def headings(self, word):
        pass
    
class UpperWordFormatter(WordFormatter):
    def headings(self, word):
        print(f'{word.upper()}')
        
class HTMLWordFormatter(WordFormatter):
    def headings(self, word):
        print(f'<html>{word}</html>')
        
class CSVWordFormatter(WordFormatter):
    def headings(self, word):
        print(f',{word},')
        
def create_formatter(name):
    if name == 'upper':
        formatter = UpperWordFormatter
    elif name == 'csv':
        formatter = CSVWordFormatter
    elif name == 'html':
        formatter = HTMLWordFormatter
    else:
        raise ValueError(f'Unknown format {name}')
    return formatter()

In [27]:
formatter = create_formatter('upper')

In [28]:
formatter

In [29]:
print_word('hi', formatter)

HI


## create a dictionary instead
### problem: how to get things into dictionary

In [30]:
from abc import ABC, abstractmethod

def print_word(word, formatter):  
    if not isinstance(formatter, WordFormatter):
        raise TypeError('formatter must be a WordFormatter')
        
    formatter.headings(word)

class WordFormatter(ABC):
    @abstractmethod
    def headings(self, word):
        pass
    
class UpperWordFormatter(WordFormatter):
    def headings(self, word):
        print(f'{word.upper()}')
        
class HTMLWordFormatter(WordFormatter):
    def headings(self, word):
        print(f'<html>{word}</html>')
        
class CSVWordFormatter(WordFormatter):
    def headings(self, word):
        print(f',{word},')
        
_formatters = {
    'upper': UpperWordFormatter,
    'csv': CSVWordFormatter,
    'html': HTMLWordFormatter
}
        
def create_formatter(name):
    formatter = _formatters.get(name)
    if not formatter:
        raise ValueError(f'Unknown format {name}')
    return formatter()

In [31]:
formatter = create_formatter('upper')

In [32]:
print_word('hi', formatter)

HI


## add the class to the dictionary when creating class
### problem: but have to do for each class

In [33]:
from abc import ABC, abstractmethod

def print_word(word, formatter):  
    if not isinstance(formatter, WordFormatter):
        raise TypeError('formatter must be a WordFormatter')
        
    formatter.headings(word)

class WordFormatter(ABC):
    @abstractmethod
    def headings(self, word):
        pass
    
_formatters = {}
def register_formatter(name, cls):
    _formatters[name] = cls
    
class UpperWordFormatter(WordFormatter):
    def headings(self, word):
        print(f'{word.upper()}')
        
register_formatter('upper', UpperWordFormatter)
        
class HTMLWordFormatter(WordFormatter):
    def headings(self, word):
        print(f'<html>{word}</html>')
        
register_formatter('html', HTMLWordFormatter)
        
class CSVWordFormatter(WordFormatter):
    def headings(self, word):
        print(f',{word},')
        
register_formatter('csv', CSVWordFormatter)
        
        
def create_formatter(name):
    formatter = _formatters.get(name)
    if not formatter:
        raise ValueError(f'Unknown format {name}')
    return formatter()

In [34]:
formatter = create_formatter('upper')

In [35]:
print_word('hi', formatter)

HI


## use metaclass to solve this issue
### injecting a little processing when type creation

In [39]:
from abc import ABC, abstractmethod

def print_word(word, formatter):  
    if not isinstance(formatter, WordFormatter):
        raise TypeError('formatter must be a WordFormatter')
        
    formatter.headings(word)
    
_formatters = {}

class WordMeta(type):
    def __init__(cls, clsname, bases, methods):
        super().__init__(clsname, bases, methods)
        if hasattr(cls, 'name'):
            _formatters[cls.name] = cls

class WordFormatter(metaclass=WordMeta):
    @abstractmethod
    def headings(self, word):
        pass
    
class UpperWordFormatter(WordFormatter):
    name = 'upper'
    def headings(self, word):
        print(f'{word.upper()}')
        
        
class HTMLWordFormatter(WordFormatter):
    name = 'html'
    def headings(self, word):
        print(f'<html>{word}</html>')
        
        
class CSVWordFormatter(WordFormatter):
    name = 'csv'
    def headings(self, word):
        print(f',{word},')
              
        
def create_formatter(name):
    formatter = _formatters.get(name)
    if not formatter:
        raise ValueError(f'Unknown format {name}')
    return formatter()

In [40]:
formatter = create_formatter('upper')

In [41]:
print_word('hi', formatter)

HI
